# This is a prototype notebook.  Use at your own risk

In [1]:
%run ../OpenMSI_Arrayed_Analysis_Tools/Arrayed_Analysis_Tools_python_2_or_3.ipy

Completed loading OpenMSI Arrayed Analysis Toolkit


In [2]:
#Optional: Load an ArrayedImage from a pickle file. This way you can work off-line
filename="../OpenMSI_Arrayed_Analysis_Tools/demo_spot_save.pkl"
import pickle
img=pickle.load(open(filename,"rb"))
print(img)

ArrayedImage based on /project/projectdirs/openmsi/omsi_data_private/bpb/20120913_nimzyme.h5
Ions loaded: [1241.2, 917.45, 817.6]
# of spot locations defined: 384
# of spot pixel masks defined: None


In [3]:
coords  = img.generateSpotList()[0]


384 spots generated. number of spots with N pixels:{10: 8, 11: 105, 12: 106, 13: 122, 14: 43}


In [6]:
import requests
client = requests.Session()
import getpass
def authenticate_user(client,username):
    password = getpass.getpass()
    authURL = 'https://openmsi.nersc.gov/openmsi/client/login/'
    # Retrieve the CSRF token first
    client.get(authURL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    login_data = dict(username=username, password=password, csrfmiddlewaretoken=csrftoken, next='/')
    r = client.post(authURL, data=login_data, headers=dict(Referer=authURL))
    return client
client = authenticate_user(client,'bpb') # put your username here in the quotes


········


In [7]:
payload = {'file':img.filename,
          'expIndex':img.expIndex,'dataIndex':img.dataIndex,'qspectrum_viewerOption':'0',
          'qslice_viewerOption':'0',
          'col':0,'row':0,
          'findPeak':'0','format':'JSON'}
url = 'https://openmsi.nersc.gov/openmsi/qmz'
r = client.get(url,params=payload)
data = json.loads(r.text)
mz = np.asarray(data[u'values_spectra'])

In [ ]:
len(mz)

In [8]:
payload = {'file':img.filename,
              'expIndex':img.expIndex,'dataIndex':img.dataIndex,'qspectrum_viewerOption':'0',
              'qslice_viewerOption':'0',
              'col':'[0,1,2]','row':'[0,1,2]',
              'findPeak':'0','format':'JSON'}
url = 'https://openmsi.nersc.gov/openmsi/qspectrum'
r = client.get(url,params=payload)
data = json.loads(r.text)
intensity = data[u'spectrum']

In [10]:
len(intensity)

3

In [9]:
len(intensity[0])

37805

In [12]:
img.generateSpotList()[0]

384 spots generated. number of spots with N pixels:{10: 8, 11: 105, 12: 106, 13: 122, 14: 43}


array([[ 9,  7],
       [ 9,  8],
       [ 9,  9],
       [10,  7],
       [10,  8],
       [10,  9],
       [10, 10],
       [11,  7],
       [11,  8],
       [11,  9],
       [11, 10],
       [12,  8]])

In [22]:
payload = {'file':img.filename,
              'expIndex':img.expIndex,'dataIndex':img.dataIndex,'qspectrum_viewerOption':'0',
              'qslice_viewerOption':'0',
              'findPeak':'0','format':'JSON'}
url = 'https://openmsi.nersc.gov/openmsi/qspectrum'
spectra_list = []
spotLocations = img.spotLocations
df = {}
df['coords'] = []
df['spectra'] = []
for i,coord in enumerate(img.generateSpotList()[:10]):
    print(i)
    payload['col'] = '[' + ','.join([str(c[1]) for c in coord]) + ']'
    payload['row'] = '[' + ','.join([str(c[0]) for c in coord]) + ']'
    r = client.get(url,params=payload)
    data = json.loads(r.text)
    df['coords'].append(coords)
    df['spectra'].append(data)
        

384 spots generated. number of spots with N pixels:{10: 8, 11: 105, 12: 106, 13: 122, 14: 43}
0
1
2
3
4
5
6
7
8
9


In [20]:
# ','.join(
','.join([c[1] for c in coord])


TypeError: sequence item 0: expected str instance, numpy.int64 found

In [ ]:
len(spectra_list[0])

In [ ]:
len(spectra_list)

In [ ]:
[c[1] for c in coord]

In [ ]:
[c[0] for c in coord]